In [8]:
%pip install llmlingua
%pip install openai
%pip install torch torchvision torchaudio --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import openai

In [14]:
import torch
print(torch.__version__)
print(torch.backends.mps.is_available())

if torch.backends.mps.is_available():
  device = torch.device("mps")
else:
  device = torch.device("cpu")

# Example tensor operations on MPS
x = torch.tensor([1.0, 2.0, 3.0], device=device)
y = torch.tensor([4.0, 5.0, 6.0], device=device)
z = x + y
print(z)

2.4.0
True
tensor([5., 7., 9.], device='mps:0')


In [11]:
api_key =

In [22]:
from llmlingua import PromptCompressor
from openai import OpenAI

# llm_lingua = PromptCompressor(
#     model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
#     use_llmlingua2=True, # Whether to use llmlingua-2
# )

## Use LLMLingua-2-small model
# 
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    use_llmlingua2=True, # Whether to use llmlingua-2
    device_map='mps'
)

In [23]:
prompt = """
Can you determine if I should ask the following question to the math teacher, the social studies teacher, or neither.

the answer should be {'result': ${math|socail|neither}}
"""
compressed_prompt = llm_lingua.compress_prompt(prompt, rate=0.4, force_tokens = ['\n', '?'])

In [26]:
print('original prompt length: ', len(prompt))
print('compressed prompt length: ',len(compressed_prompt['compressed_prompt']))
print(compressed_prompt['compressed_prompt'])


original prompt length:  176
compressed prompt length:  84

 determine question math social studies 
 
 answer result ' math | socail neither 



In [27]:
client = OpenAI(api_key=api_key)

In [28]:
completion1 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": "What were the main causes and consequences of the Industrial Revolution?"}
  ]
)

In [29]:
print(completion1.choices[0].message)

ChatCompletionMessage(content="{'result': 'social'}", role='assistant', function_call=None, tool_calls=None)


In [30]:

completion2 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": compressed_prompt['compressed_prompt']},
    {"role": "user", "content": "What were the main causes and consequences of the Industrial Revolution?"}
  ]
)

In [31]:
print(completion2.choices[0].message)

ChatCompletionMessage(content='social studies', role='assistant', function_call=None, tool_calls=None)
